In [ ]:
import torchvision
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import glob

from rembg import remove

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Let's forcast the trending colors in fashion for Fall and Winter 2025 by using a clustering algorthim to analyze runway show images.

All photos taken from https://nowfashion.com/

Steps:
1. Read in image
2. Remove image background so only model is present (this is to avoid run way setting from influencing color   analysis)
3. For each image, apply K-means to RGB color data to identify 6 clusters. Each cluster centroid will correspond to a dominant color in the image. One of the 6 centroids will correspond to the color of the mask applied to the background and will be removed.
4. Aggregate all colors identified in step 3. and reapply K-means to identify main color scheme across all images


In [ ]:
def find_colors(im_name, plot = False):
    # load image
    image = Image.open(im_name).convert('RGB')

    # Removing the background from the given Image
    # and shape into 2d array
    im_no_background = remove(image, bgcolor=(255, 255, 255, 255))
    cropped_image_tensor = torchvision.transforms.functional.to_tensor(
        im_no_background)
    h, w, c = cropped_image_tensor.permute(2, 1, 0).shape
    im2d = cropped_image_tensor.permute(
        2, 1, 0).reshape(h*w, c)

    #run kmeans
    kmeans = KMeans(n_clusters=6, n_init=10)
    s = kmeans.fit(im2d)

    labels = kmeans.labels_
    labels = list(labels)
    centroid = kmeans.cluster_centers_

    #one centroid will include mask color as a color
    #with values approximately equal to [1,1,1,1]
    #remove that one by finding element closest to sum([1,1,1,1])
    good_centroids = []
    for i in range(6):
        if round(np.sum(centroid[i])) != 4:
            good_centroids.append(i)

    #find percent representation of each color
    percent = []
    for i in good_centroids:
        j = labels.count(i)
        j = j/(len(labels))
        percent.append(j)

    if plot == True:
        plt.pie(percent, colors=np.array(centroid[good_centroids]),
            labels=np.arange(len(centroid[good_centroids])))
        plt.show()
        plt.imshow(image)
        plt.show()

    return percent, centroid[good_centroids]

In [5]:
images = glob.glob("all_images/*/*")

Demonstrate image background removal

In [ ]:
# load image
image = Image.open(images[0]).convert('RGB')

# Removing the background from the given Image
# and shape into 2d array
im_no_background = remove(image, bgcolor=(255, 255, 255, 255))
plt.imshow(im_no_background)

In [ ]:
centroids = []
percents = []
for i in images:
    percent, centroid = find_colors(i)
    percents.append(percent)
    centroids.append(centroid)

In [ ]:
tensor = torchvision.transforms.functional.to_tensor(
    np.concatenate(centroids))

# h, w, c = tensor.permute(1,0).shape
tensor_2d = tensor[0]

kmeans = KMeans(n_clusters=10, n_init=10)
s = kmeans.fit(tensor_2d)

labels = kmeans.labels_
labels = list(labels)
centroid = kmeans.cluster_centers_

# find percent representation of each color
percent = []
for i in range(len(centroid)):
    j = labels.count(i)
    j = j/(len(labels))
    percent.append(j)

plt.pie(percent, colors=centroid)
plt.show()